In [ ]:
# Databricks notebook source

from pyspark.sql.functions import current_timestamp, col
from pyspark.sql.utils import AnalysisException

# --- Unity Catalog Configuration ---catalog_name = "main_catalog"schema_name = "etl_demo"input_table_name = "orders_raw"output_table_name = "orders_transformed"
# Use the catalog and schemaspark.sql(f"USE CATALOG {catalog_name}")spark.sql(f"USE SCHEMA {schema_name}")
try:    print(f"Reading raw orders data from {catalog_name}.{schema_name}.{input_table_name}...")    # Read the raw data from the Delta table    df = spark.read.table(f"{catalog_name}.{schema_name}.{input_table_name}")    print(f"Successfully read {df.count()} rows from {catalog_name}.{schema_name}.{input_table_name}.")
    # --- Data Quality Checks ---    # Drop rows with null values in critical columns    critical_columns = ["order_id", "user_id", "product_id", "quantity", "price"]    initial_count = df.count()    df_cleaned = df.na.drop(subset=critical_columns)    cleaned_count = df_cleaned.count()
    if initial_count > cleaned_count:        print(f"Dropped {initial_count - cleaned_count} rows due to null values in critical columns.")    else:        print("No null values found in critical columns.")
    # --- Transformations ---    print("Performing transformations...")    df_transformed = df_cleaned.withColumn("processing_timestamp", current_timestamp())    print("Transformations complete.")
    # --- Write Transformed Data ---    print(f"Writing transformed orders data to {catalog_name}.{schema_name}.{output_table_name}...")    df_transformed.write.format("delta").mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.{output_table_name}")    print(f"Successfully transformed orders data and saved to {catalog_name}.{schema_name}.{output_table_name}.")
except AnalysisException as e:    print(f"Error reading or writing Delta table: {e}")    print("Please ensure the input table exists and has valid Delta table data.")except Exception as e:    print(f"An unexpected error occurred: {e}")